In [1]:
# w-> window size of the motif
# t -> size of each sequence, all sequences have the same length
# n -> number of sequences

seqs = ['GTAAACAATATTTATAGC', 'AAAATTTACCTTAGAAGG', 'CCGTACTGTCAAGCGTGG', 'TGAGTAAACGACGTCCCA', 'TACTTAACACCCTGTCAA']

In [61]:
class PWM:
    def __init__(self, seqs: list[str], seq_type: str = 'DNA', psuedo: int =1) -> None:
        self.seqs = seqs
        self.seq_type = seq_type
        self.pseudo = psuedo
        self.pwm = self.pwm()

    def alfabeto(self) -> list[str]:
        '''
        Esta função recebe o tipo de uma sequência e devolve o seu respetivo alfabeto

        Parameters
        ---------
        tipo_seq:str
            Tipo válido de uma sequência biológica. Pode assumir os valores de DNA, RNA, protein,
            proteína ou prot

        Returns
        -------
        list(str)
            Composto pelo tipo de sequência e o seu respetivo alfabeto
        
        Raises
        ------
        ValueError
            Quando o tipo de sequência fornecido não é um tipo de sequência biológica válida
        '''
        assert (type(self.seq_type)==str), 'O tipo da sequência deve ser fornecida em formato de string'
        if self.seq_type.upper() == "DNA":
            DNA = "ACGT"
            return DNA
        elif self.seq_type.upper() == "RNA":
            RNA = "ACGU"
            return RNA
        elif self.seq_type.upper() in ["PROTEIN", "PROTEINA", "PROT"]:
            protein = "ABCDEFGHIKLMNPQRSTVWYZ"
            return protein
        else:
            raise ValueError(f"O tipo {self.seq_type} não é válido. Os tipos de cadeia válida são: DNA, RNA ou proteina")


    def pwm(self) -> list[dict[str, float]]:
        '''
        Esta função recebe uma lista de sequências alinhadas e devolve uma matriz desingada de position weigth matrix (PWM), onde se representa
        a probabilidade de cada letra de um dado alfabeto biológico estar presente numa determinada posição.

        Parameters
        ----------
        list_seqs:list(str)
            Contêm um conjunto de sequências alinhadas em formato de string

        tipo_seq:str
            Tipo da sequencia que se pretende realiazar a PWM. Pode assumir o valor de DNA, RNA, protein, proteína ou prot

        pseudo: int ou float
            Valor a utilizar para realizar a pseudocontagem. Por padrão o valor é 0

        Returns
        -------
        PWM:list(dict)
            Valor da probabilidade de cada letra do alfabeto estar presente em cada posição das sequências
        
        Raises
        ------
        AssertionError
            Quando um elemento de uma das sequências fornecidas não está presente no alfabeto utilizado

            Quando o tamanho das sequências fornecidas é diferente
        '''
        assert type(self.seqs)==list, 'As sequencias devem ser fornecidas em formato de lista'
        assert all(len(self.seqs[0])==len(seq)for seq in self.seqs), 'As sequencias devem possuir o mesmo tamanho'
        
        alfa = self.alfabeto()

        for seq in self.seqs:
            for pos, elem in enumerate(seq):
                assert elem in alfa, f"Na sequencia {seq}, o elemento {elem} na posição {pos + 1} não se encontra no alfabeto. Verifica se o elemento está em maiusculas"


        PWM = []

        for coluna in list(zip(*self.seqs)):
            contagem = {elem:(coluna.count(elem) + self.pseudo) / (len(self.seqs) + len(alfa) * self.pseudo) for elem in alfa}
            PWM.append(contagem)


        return PWM
    

    def prob_seq(self, seq: str):
        '''
        Esta função calcula a probabilidade de uma determinada sequencia ter sido gerada pela PWM

        Parameters
        ---------
        seq:str
            Sequência que se pretende calcular a probabilidade de ter sido gerada pela PWM
        
        PWM:list(dict[str,floar])
            Corresponde às probabilidades de cada elemento aparecer nas várias posições da sequência

        Returns
        -------
        prod:float
            Probabilidade de a sequencia fornecida ter sido gerada pela PWM

        Raises
        ------
        AssertionError
            Quando o tamanho da sequência não é igual ao tamanho da PWM

        '''
        assert len(seq) <= len(self.pwm), f"A sequência {seq} não é do mesmo tamanho que a PWM"

        prod = 1
        for I, elem in enumerate(seq):
            prod = prod * self.pwm[I][elem]
        return prod
    

In [59]:
class Gibbs (PWM):
    def __init__(self, list_seqs: list[str], w: int, seq_type='DNA', pseudo=1):
        self.seqs = list_seqs
        self.w = w
        self.seq_type = seq_type
        self.pseudo = pseudo
        
    
    def random_init_pos(self) -> list[int]:
        import random
        assert type(self.seqs) == list, 'As sequencias devem ser fornecidas em formato de lista'
        t = len(self.seqs[0])  # length of each seq
        n = len(self.seqs)  # number of seqs
        max_pos = t - self.w  # last possible position the use to form a seq with length w
        pos = [random.randint(0, max_pos) for _ in range(n)]  # choose random positions between 0 and max_pos

        return pos
    

    def random_seq(self) -> int:
        import random
        n = len(self.seqs)
        return random.randint(0, n - 1)
    

    def new_seqs(self, pos: list[int], seq_idx: int):
        assert type(pos) == list and type(seq_idx) == int
        novas_seqs = [self.seqs[idx][P:P + self.w] for idx, P in enumerate(pos) if idx != seq_idx]  # create the motifs with the size w
        return novas_seqs, seqs[seq_idx]
    
    

    def prob(self, pos: list[int], seq_idx: int, first=True):
        if first:
            possible_motiff, seq_excluded = self.new_seqs(pos, seq_idx)

        else:
            possible_motiff = [seq[pos[idx]: pos[idx] + self.w] for idx, seq in enumerate(self.seqs) if idx != seq_idx]
            seq_excluded = self.seqs[seq_idx]

        #super().__init__(possible_motiff, self.seq_type, self.pseudo)
        t = len(self.seqs[0])
        max_pos = t - self.w
        seqs_pwm = PWM(possible_motiff, self.seq_type, self.pseudo).pwm()
        prob = []

        for I in range(max_pos + 1):
            seq = seq_excluded[I:I + self.w]
            prob.append(self.prob_seq(seq, seqs_pwm))

        return [prob[I] / sum(prob) for I in range(len(prob))]


    def roulette_wheel(self, probs: list[float]):
        import random
        assert (round(sum(probs), 0) == 1)
        pos = [I for I in range(len(probs))]

        return random.choices(pos, probs, k=1)


    def score(self, off: list[int]) -> int:
        alfa=self.alfabeto()
        snips = [self.seqs[I][p:p + self.w] for I, p in enumerate(off)]
        return sum(max({x: s.count(x) for x in alfa}.values()) for s in zip(*snips))


    def gibbs_sampling(self):
        pos = self.random_init_pos()
        first_run, max_sc, contador = True, 0, 0

        while True:
            seq_idx = self.random_seq()

            if first_run:
                probs, first_run = self.prob(pos, seq_idx), False
            
            else: probs = self.prob(pos, seq_idx, first=False)

            pos[seq_idx] = self.roulette_wheel(probs)[0]

            if self.score(pos) > max_sc:
                best_off, max_sc, contador = pos.copy(), self.score(pos), 0 

            else:
                contador += 1
                if contador >=50: break

        return best_off, max_sc
        

In [29]:
Gibbs(seqs,8).random_seq()

1

In [63]:
Gibbs(seqs,8).prob([6, 3, 9, 9, 4],1,first=False)

TypeError: 'list' object is not callable

In [25]:
Gibbs(seqs,8).gibbs_sampling()

TypeError: 'list' object is not callable

In [20]:
a=Gibbs(seqs,8)

In [15]:
a.gibbs_sampling()

TypeError: PWM.pwm() takes 1 positional argument but 2 were given

In [361]:
class Gibbs:
    def __init__(self, list_seqs: list[str], w: int, seq_type='DNA', pseudo=1):
        self.seqs = list_seqs
        self.w = w
        self.seq_type = seq_type
        self.pseudo = pseudo
        
    
    def random_init_pos(self) -> list[int]:
        import random
        assert type(self.seqs) == list, 'As sequencias devem ser fornecidas em formato de lista'
        t = len(self.seqs[0])  # length of each seq
        n = len(self.seqs)  # number of seqs
        max_pos = t - self.w  # last possible position the use to form a seq with length w
        pos = [random.randint(0, max_pos) for _ in range(n)]  # choose random positions between 0 and max_pos

        return pos
    

    def random_seq(self) -> int:
        import random
        n = len(self.seqs)
        return random.randint(0, n - 1)
    

    def new_seqs(self, pos: list[int], seq_idx: int):
        assert type(pos) == list and type(seq_idx) == int
        novas_seqs = [self.seqs[idx][P:P + self.w] for idx, P in enumerate(pos) if idx != seq_idx]  # create the motifs with the size w
        return novas_seqs, seqs[seq_idx]
    

    def alfabeto(self) -> list[str]:
        '''
        Esta função recebe o tipo de uma sequência e devolve o seu respetivo alfabeto

        Parameters
        ---------
        tipo_seq:str
            Tipo válido de uma sequência biológica. Pode assumir os valores de DNA, RNA, protein,
            proteína ou prot

        Returns
        -------
        list(str)
            Composto pelo tipo de sequência e o seu respetivo alfabeto
        
        Raises
        ------
        ValueError
            Quando o tipo de sequência fornecido não é um tipo de sequência biológica válida
        '''
        assert (type(self.seq_type)==str), 'O tipo da sequência deve ser fornecida em formato de string'
        if self.seq_type.upper() == "DNA":
            DNA = "ACGT"
            return DNA
        elif self.seq_type.upper() == "RNA":
            RNA = "ACGU"
            return RNA
        elif self.seq_type.upper() in ["PROTEIN", "PROTEINA", "PROT"]:
            protein = "ABCDEFGHIKLMNPQRSTVWYZ"
            return protein
        else:
            raise ValueError(f"O tipo {self.seq_type} não é válido. Os tipos de cadeia válida são: DNA, RNA ou proteina")


    def pwm(self,seqs: list[str]) -> list[dict[str, float]]:
        '''
        Esta função recebe uma lista de sequências alinhadas e devolve uma matriz desingada de position weigth matrix (PWM), onde se representa
        a probabilidade de cada letra de um dado alfabeto biológico estar presente numa determinada posição.

        Parameters
        ----------
        list_seqs:list(str)
            Contêm um conjunto de sequências alinhadas em formato de string

        tipo_seq:str
            Tipo da sequencia que se pretende realiazar a PWM. Pode assumir o valor de DNA, RNA, protein, proteína ou prot

        pseudo: int ou float
            Valor a utilizar para realizar a pseudocontagem. Por padrão o valor é 0

        Returns
        -------
        PWM:list(dict)
            Valor da probabilidade de cada letra do alfabeto estar presente em cada posição das sequências
        
        Raises
        ------
        AssertionError
            Quando um elemento de uma das sequências fornecidas não está presente no alfabeto utilizado

            Quando o tamanho das sequências fornecidas é diferente
        '''
        assert type(seqs)==list, 'As sequencias devem ser fornecidas em formato de lista'
        assert all(len(seqs[0])==len(seq)for seq in seqs), 'As sequencias devem possuir o mesmo tamanho'
        
        alfa = self.alfabeto()

        for seq in seqs:
            for pos, elem in enumerate(seq):
                assert elem in alfa, f"Na sequencia {seq}, o elemento {elem} na posição {pos + 1} não se encontra no alfabeto. Verifica se o elemento está em maiusculas"


        PWM = []

        for coluna in list(zip(*seqs)):
            contagem = {elem:(coluna.count(elem) + self.pseudo) / (len(seqs) + len(alfa) * self.pseudo) for elem in alfa}
            PWM.append(contagem)


        return PWM
    

    def prob_seq(self,seq: str, PWM: list[dict[str, float]]):
        '''
        Esta função calcula a probabilidade de uma determinada sequencia ter sido gerada pela PWM

        Parameters
        ---------
        seq:str
            Sequência que se pretende calcular a probabilidade de ter sido gerada pela PWM
        
        PWM:list(dict[str,floar])
            Corresponde às probabilidades de cada elemento aparecer nas várias posições da sequência

        Returns
        -------
        prod:float
            Probabilidade de a sequencia fornecida ter sido gerada pela PWM

        Raises
        ------
        AssertionError
            Quando o tamanho da sequência não é igual ao tamanho da PWM

        '''
        assert len(seq) <= len(PWM), f"A sequência {seq} não é do mesmo tamanho que a PWM"

        prod = 1
        for I, elem in enumerate(seq):
            prod = prod * PWM[I][elem]
        return prod
    

    def prob(self, pos: list[int], seq_idx: int, first=True):
        if first:
            possible_motiff, seq_excluded = self.new_seqs(pos, seq_idx)

        else:
            possible_motiff = [seq[pos[idx]: pos[idx] + self.w] for idx, seq in enumerate(self.seqs) if idx != seq_idx]
            seq_excluded = self.seqs[seq_idx]

        t = len(self.seqs[0])
        max_pos = t - self.w
        PWM = self.pwm(possible_motiff)
        prob = []

        for I in range(max_pos + 1):
            seq = seq_excluded[I:I + self.w]
            prob.append(self.prob_seq(seq, PWM))

        return [prob[I] / sum(prob) for I in range(len(prob))]


    def roulette_wheel(self, probs: list[float]):
        import random
        assert (round(sum(probs), 0) == 1)
        pos = [I for I in range(len(probs))]

        return random.choices(pos, probs, k=1)


    def score(self, off: list[int]) -> int:
        alfa=self.alfabeto()
        snips = [self.seqs[I][p:p + self.w] for I, p in enumerate(off)]
        return sum(max({x: s.count(x) for x in alfa}.values()) for s in zip(*snips))


    def gibbs_sampling(self):
        pos = self.random_init_pos()
        first_run, max_sc, contador = True, 0, 0

        while True:
            seq_idx = self.random_seq()

            if first_run:
                probs, first_run = self.prob(pos, seq_idx), False
            
            else: probs = self.prob(pos, seq_idx, first=False)

            pos[seq_idx] = self.roulette_wheel(probs)[0]

            if self.score(pos) > max_sc:
                best_off, max_sc, contador = pos.copy(), self.score(pos), 0 

            else:
                contador += 1
                if contador >=50: break

        return best_off, max_sc
        

In [429]:
Gibbs(seqs,8).gibbs_sampling()

([9, 3, 4, 10, 1], 28)

In [ ]:
def gibbs_sampling_2(list_seqs: list[str], w: int, seq_type='DNA', pseudo=1):
    pos = random_init_pos(list_seqs, w)
    seq_idx = random_seq(list_seqs)

    # first run
    probs = prob_2(list_seqs, w, pos, seq_idx, seq_type, pseudo)
    posicao = roulette_wheel_2(probs)
    pos[seq_idx] = posicao
    max_sc = score(pos, seqs, w)
    best_off = pos.copy()

    # continuing till there's no change in the score after 10 iterations

    contador = 0

    while True:
        seq_idx = random_seq(list_seqs)
        probs = prob(list_seqs, w, pos, seq_idx, seq_type=seq_type, first=False)
        posicao = roulette_wheel_2(probs)
        pos[seq_idx] = posicao

        if score(pos, list_seqs, w) > max_sc:
            best_off = pos.copy()
            max_sc = score(pos, seqs, w)
            contador = 0

        else:
            contador += 1

        if contador >= 100:
            break

    return best_off, max_sc


def gibbs_sampling(list_seqs: list[str], w: int, seq_type='DNA', pseudo=1):
    pos = random_init_pos(list_seqs, w)
    seq_idx = random_seq(list_seqs)

    # first run
    probs = prob(list_seqs, w, pos, seq_idx, seq_type, pseudo)
    posicao = roulette_wheel(probs)
    pos[seq_idx] = posicao[0]
    max_sc = score(pos, seqs, w)
    best_off = pos.copy()

    # continuing till there's no change in the score after 10 iterations

    contador = 0

    while True:

        seq_idx = random_seq(list_seqs)
        probs = prob(list_seqs, w, pos, seq_idx, seq_type=seq_type, first=False)
        posicao = roulette_wheel(probs)
        pos[seq_idx] = posicao[0]

        if score(pos, list_seqs, w) > max_sc:
            best_off = pos.copy()
            max_sc = score(pos, seqs, w)
            contador = 0

        else:
            contador += 1

        if contador >= 1000:
            break

    return best_off, max_sc
